In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import dunestyle.matplotlib as dunestyle
import pandas as pd

/home/leoperes/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


DUNE plot style enabled


In [2]:
# File path to the data file
bartol_numu_solmax = 'fmax20_0401z.sou_num'  
bartol_anumu_solmax = 'fmax20_0401z.sou_nbm'
bartol_nue_solmax = 'fmax20_0401z.sou_nue'
bartol_anue_solmax = 'fmax20_0401z.sou_nbe'

bartol_numu_solmin = 'fmin20_0401z.sou_num'
bartol_anumu_solmin = 'fmin20_0401z.sou_nbm'
bartol_nue_solmin = 'fmin20_0401z.sou_nue'
bartol_anue_solmin = 'fmin20_0401z.sou_nbe'

bartol_solmax_list = [bartol_numu_solmax, bartol_anumu_solmax, bartol_nue_solmax, bartol_anue_solmax]
bartol_solmin_list = [bartol_numu_solmin, bartol_anumu_solmin, bartol_nue_solmin, bartol_anue_solmin]

def ReadBartolTable(file_path):
    # Column names as per the format description
    columns = [
        "Energy_Center_GeV",        # Energy at the center of the bin (GeV)
        "Cos_Zenith_or_Azimuth",    # Cos(zenith) or azimuth (degrees)
        "Flux_dN/dlogE_m2_s_sr",    # Flux (dN/d(logE)) in /m²/steradian/sec
        "MC_Stat_Error",            # Monte Carlo statistical error
        "Unweighted_Events"         # Number of unweighted events in the bin
    ]

    # Load the file into a DataFrame
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # Find the index where data starts (skip lines beginning with #)
        data_start_idx = next(i for i, line in enumerate(lines) if not line.startswith('#'))

    # Read the data into a DataFrame
    df_flux = pd.read_csv(
        file_path,
        delim_whitespace=True,  # Columns are separated by whitespace
        skiprows=data_start_idx,  # Skip comment lines
        names=columns           # Assign column names
    )

    df_flux = df_flux.iloc[1:].reset_index(drop=True)

    # Display the first few rows to ensure data is loaded correctly
    #print(df_flux.head())

    # Drop the first line (row) of the DataFrame
    return df_flux

In [3]:
def E_dflux_dE_from_df(dataframe_flux):
    energy = np.array(dataframe_flux["Energy_Center_GeV"].values, dtype=float)
    flux = dataframe_flux["Flux_dN/dlogE_m2_s_sr"].values
    dflux_dE = np.array(flux, dtype=float) /  energy
    return energy, dflux_dE

In [ ]:
#flux = [ReadBartolTable(flavor_nu) for flavor_nu in bartol_solmax_list]

df_fluxes = [ReadBartolTable(bartol_flavor) for bartol_flavor in bartol_solmin_list]
cos_honda = [' 1.00',' 0.90',' 0.80',' 0.70',' 0.60', ' 0.50', ' 0.40', ' 0.30', ' 0.20', ' 0.10', ' 0.00', '-0.10', '-0.20', '-0.30', '-0.40', '-0.50', '-0.60', '-0.70', '-0.80', '-0.90', '-1.00']

# Assuming df_flux is the DataFrame containing the data
cos_values = df_fluxes[0]["Cos_Zenith_or_Azimuth"].unique()  # Unique cos(zenith) values
cos_values = np.array(cos_values)[::-1]

for id, cos_val in enumerate(cos_values):
    
    # Text lines that change with each loop
    line1 = rf"average flux in [cosZ ={cos_honda[id+1]} -- {cos_honda[id]}, phi_Az =   0 -- 360]"
    line2 = " Enu(GeV)   NuMu       NuMubar    NuE        NuEbar  (m^2 sec sr GeV)^-1"
    # Filter the DataFrame for the current cos(zenith) value
    groups_df = [df_flux[df_flux["Cos_Zenith_or_Azimuth"] == cos_val] for df_flux in df_fluxes]
        
    results = [E_dflux_dE_from_df(group) for group in groups_df]
    energies = [result[0] for result in results]
    fluxes = [result[1] for result in results]
           
    # Stack the arrays column-wise
    stacked_arrays = np.column_stack((energies[0], fluxes[0], fluxes[1], fluxes[2], fluxes[3]))
    
    # Open the file in append mode
    with open('bar_solmin_hondaformat.d', 'a') as f:
        # Write the custom text lines
        f.write(line1 + '\n')
        f.write(line2 + '\n')

        # Write the 5 NumPy arrays (each as a column)
        stacked_arrays = np.column_stack((energies[0], fluxes[0], fluxes[1], fluxes[2], fluxes[3]))  # Stack arrays column-wise
        np.savetxt(f, stacked_arrays, fmt=' %.4E', delimiter=" ")
        
print("bar_solmin_hondaformat.d")

bar_solmax_hondaformat.d


/tmp/ipykernel_13067/434497588.py:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_flux = pd.read_csv(
/tmp/ipykernel_13067/434497588.py:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_flux = pd.read_csv(
/tmp/ipykernel_13067/434497588.py:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_flux = pd.read_csv(
/tmp/ipykernel_13067/434497588.py:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_flux = pd.read_csv(


In [42]:
cos_honda

'[1.00 0.90 0.80 0.70 0.60 0.50 0.40 0.30 0.20 0.10 0.00 -0.10 -0.20 -0.30\n -0.40 -0.50 -0.60 -0.70 -0.80 -0.90 -1.00]'

In [ ]:
['1.00','0.90','0.80','0.70','0.60', '0.50', '0.40', '0.30', '0.20', '0.10', '0.00', '-0.10', '-0.20', '-0.30', '-0.40', '-0.50', '-0.60', '-0.70', '-0.80', '-0.90', '-1.00']